# Microsoft (MSFT) ¿En qué meses alcanzó precios más altos y mas bajos?

### 1. Instalación de librerias

*   yfinance → para traer datos de Yahoo Finance
*   plotly → para graficar los datos de manera interactiva

In [1]:
!pip install yfinance plotly pandas

### 2. Importación de librerías

In [2]:
import yfinance as yf
import pandas as pd
import plotly.express as px

### 3. Descargamos datos de Microsoft (MSFT)

Ajustamos fechas el rango a los ultimos meses del año y descargamos datos de cierre diario de MSFT con la variable (data).

1.   **Download→** descarga los datos históricos de un ticker.

2.   **Parámetros:**

*   "MSFT" → ticker de Microsoft.
*   start y end → rango de fechas.

3.   **["Close"]→** precio de cierre diario.

Resultado: un Series de Pandas (una sola columna con índice de fechas).


In [28]:
data = yf.download("MSFT", start="2023-01-01", end="2025-08-31")["Close"]


/tmp/ipython-input-3500486354.py:1: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


### 4. Agrupamos por mes (promedio, máximo y mínimo de cada uno)

*   **.resample("M")→**
Agrupa los datos por un intervalo de tiempo.
"M" significa fin de mes.
Ejemplo: todos los precios de enero se agrupan juntos, luego febrero, etc.

*   **.agg(["max", "min"])→**
Aplica funciones de agregación sobre cada grupo.
"max" → el precio máximo del mes.
"min" → el precio mínimo del mes.
Devuelve un DataFrame con dos columnas: max y min.

In [29]:
monthly = data.resample("ME").agg(["max", "min"])

### 5. Identificamos el mes con el precio más alto y el más bajo

*   **monthly['MSFT_max'] →** selecciona la columna que contiene los valores máximos de MSFT en cada mes.

*   **.idxmax() →** devuelve el índice (fecha) donde ese valor máximo ocurrió.

In [30]:
max_month = monthly[('MSFT', 'max')].idxmax()
min_month = monthly[('MSFT', 'min')].idxmin()

print(f"Mes con el precio más alto: {max_month.strftime('%B %Y')}")
print(f"Mes con el precio más bajo: {min_month.strftime('%B %Y')}")

Mes con el precio más alto: August 2025
Mes con el precio más bajo: January 2023


In [31]:
# Convierte el DataFrame de formato ancho (con columnas max, min) a formato largo
monthly_long = monthly.stack(future_stack=True).reset_index()

# Renombra las columnas para que quede más legible
monthly_long.columns = ["Date", "Stat", "MSFT"]

# Verificación
print(monthly_long.columns)
display(monthly_long.head())

Index(['Date', 'Stat', 'MSFT'], dtype='object')


,Date,Stat,MSFT
0,2023-01-31,max,242.818909
1,2023-01-31,min,217.525253
2,2023-02-28,max,266.312164
3,2023-02-28,min,244.466888
4,2023-03-31,max,282.801575


### 6. Visualización con Plotly

Se crea un gráfico de líneas que muestra la evolución de los precios máximos y mínimos mensuales de MSFT en un gráfico interactivo.

*   **monthly →** DataFrame con los precios máximos y mínimos por mes.

*   x=monthly.index → eje X = fechas (meses).

*   y=["MSFT_max","MSFT_min"] → eje Y = dos líneas (máximos y mínimos).

*   **title →** título del gráfico.

*   **labels →** renombra ejes y leyenda para que se entiendan mejor.

*   **fig.show()→** Muestra el gráfico interactivo en Colab (puedes mover, hacer zoom, ver valores).


In [32]:
monthly_stacked = monthly.stack().reset_index()
monthly_stacked.columns = ['Date', 'Type', 'Price']

fig = px.line(
    monthly_stacked,
    x='Date',
    y='Price',
    color='Type',
    title="Precios mensuales máximos y mínimos de Microsoft (MSFT)",
    labels={"Price": "Precio USD", "Date": "Fecha", "Type": "Tipo"}
)
fig.show()

/tmp/ipython-input-176558452.py:1: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.

